## Rerun imports and initialization

In [ ]:
from __future__ import annotations

import math
from collections import namedtuple
from math import cos, sin, tau

import numpy as np
import rerun as rr  # pip install rerun-sdk
import rerun.blueprint as rrb


## Helper to create the colored cube

This code exists in the `rerun.utilities` package, but is included here for context.

In [ ]:
ColorGrid = namedtuple("ColorGrid", ["positions", "colors"])


def build_color_grid(x_count=10, y_count=10, z_count=10, twist=0):
    """
    Create a cube of points with colors.

    The total point cloud will have x_count * y_count * z_count points.

    Parameters
    ----------
    x_count, y_count, z_count:
        Number of points in each dimension.
    twist:
        Angle to twist from bottom to top of the cube

    """

    grid = np.mgrid[
        slice(-x_count, x_count, x_count * 1j),
        slice(-y_count, y_count, y_count * 1j),
        slice(-z_count, z_count, z_count * 1j),
    ]

    angle = np.linspace(-float(twist) / 2, float(twist) / 2, z_count)
    for z in range(z_count):
        xv, yv, zv = grid[:, :, :, z]
        rot_xv = xv * cos(angle[z]) - yv * sin(angle[z])
        rot_yv = xv * sin(angle[z]) + yv * cos(angle[z])
        grid[:, :, :, z] = [rot_xv, rot_yv, zv]

    positions = np.vstack([xyz.ravel() for xyz in grid])

    colors = np.vstack(
        [
            xyz.ravel()
            for xyz in np.mgrid[
                slice(0, 255, x_count * 1j),
                slice(0, 255, y_count * 1j),
                slice(0, 255, z_count * 1j),
            ]
        ]
    )

    return ColorGrid(positions.T, colors.T.astype(np.uint8))


## Logging some data

Now we can create some data and add it to the recording.

In [ ]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time_sequence("step", t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))


## Viewing the output

The current global stream can be output to the cell using `rr.notebook_show()`

In [ ]:
rr.notebook_show()


## Adjusting the view

The  `show` method also lets you adjust properties such as width and height.

In [ ]:
rr.notebook_show(width=400, height=400)


## Stating a new recording

You can always start another recording by calling `rr.init(...)` again to reset the global stream, or alternatively creating a separate recording stream using `rr.new_recording` (discussed more below)

In [ ]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time_sequence("step", t)
    h_grid = build_color_grid(10, 3, 3, twist=twists[t])
    rr.log("h_grid", rr.Points3D(h_grid.positions, colors=h_grid.colors, radii=0.5))
    v_grid = build_color_grid(3, 3, 10, twist=twists[t])
    rr.log("v_grid", rr.Points3D(v_grid.positions, colors=v_grid.colors, radii=0.5))

rr.notebook_show()


## Using blueprints

Rerun blueprints can be used with `rr.show()`

For example, we can split the two grids into their own respective space-views.

In [ ]:
blueprint = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(name="Horizontal grid", origin="h_grid"),
        rrb.Spatial3DView(name="Vertical grid", origin="v_grid"),
        column_shares=[2,1]),
    collapse_panels=True
)

rr.notebook_show(blueprint=blueprint)


## Working with non-global streams

Sometimes it can be more explicit to work with specific (non-global recording) streams via the `new_recording` method.

In this case, remember to call `notebook_show` directly on the recording stream. As noted above, there is no way to use a bare Blueprint object in conjunction with a non-global recording.

In [ ]:
rec = rr.new_recording("rerun_example_cube_flat")

flat_grid = build_color_grid(20, 20, 1, twist=0)
rec.log("flat_grid", rr.Points3D(flat_grid.positions, colors=flat_grid.colors, radii=0.5))

bp = rrb.Blueprint(collapse_panels=True)

rec.notebook_show(blueprint=bp)
